<a href="https://colab.research.google.com/github/deenyse/VSB_ZSU/blob/main/Copy_of_UASS_5_21_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analýza větších sítí

> Cílem cvičení je vyzkoušet si analyzovat větší sítě s využitím poznatků z minulých cvičení, tj. vizualizovat síť, vlastnosti, komunity, apod.


## Část 1: Struktury v sítí

1. Sítě

  - Vyberte si alespoň 2 větší sítě [large_data.zip](https://h`omel.vsb.cz/~kud007/data/large_data.zip)

2. Analyzujte síť, výstupem by měl být dokument, kde shrnete získané poznatky

In [10]:
import random
import networkx as nx

import pandas as pd
import numpy as np

def find_components(G):
    return list(nx.connected_components(G))


def find_cliques(G, min_size=3):
    clique_nodes = set()
    for clique in nx.find_cliques(G):
        if len(clique) >= min_size:
            clique_nodes.update(clique)
    return clique_nodes


def find_stars(G, degree_percentile=80, leaf_density_threshold=0.2):
    degrees = [G.degree(n) for n in G]
    min_degree = np.percentile(degrees, degree_percentile)
    star_nodes = set()
    for node in G:
        if G.degree(node) < min_degree:
            continue
        neighbors = list(G.neighbors(node))
        if len(neighbors) < 3:
            continue
        if nx.density(G.subgraph(neighbors)) <= leaf_density_threshold:
            star_nodes.add(node)
    return star_nodes



In [11]:
def color_nodes(G, nodes, color, node_type=None):
    for n in nodes:
        if n not in G.nodes:
            continue
        G.nodes[n]["color"] = color
        if node_type:
            G.nodes[n]["type"] = node_type


def export_to_gephi(G: nx.Graph, path: str = "graph_colored.gexf"):
    nx.write_gexf(G, path)



In [25]:
# Function for network analysis
def analyze_network(file_path, network_name):
    print(f"\n=== Results for network: {network_name} ===")

    try:
        # Read the graph in GEXF format
        G = nx.read_gexf(file_path)

        # Basic metrics
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        avg_degree = sum(dict(G.degree()).values()) / num_nodes if num_nodes > 0 else 0
        components = find_components(G)
        clique_nodes = find_cliques(G, min_size=4)
        star_nodes = find_stars(G, degree_percentile=80, leaf_density_threshold=0.2)

        # Node coloring and export
        nx.set_node_attributes(G, "default", "type")
        nx.set_node_attributes(G, "gray", "color")
        color_nodes(G, clique_nodes, "#ff0000", "clique")
        color_nodes(G, star_nodes, "#00ccff", "star")
        export_to_gephi(G, f"{network_name}_colored.gexf")

        # Print results
        print(f"Number of nodes: {num_nodes}")
        print(f"Number of edges: {num_edges}")
        print(f"Average node degree: {avg_degree:.2f}")
        print(f"Number of connected components: {len(components)}")
        print(f"Number of nodes in cliques (min_size=4): {len(clique_nodes)}")
        print(f"Number of nodes in stars: {len(star_nodes)}")

    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")

# Analyze two networks
networks = [
    ("Ernon.csv", "Enron"),
    ("Facebook.csv", "Facebook")
]

for file_path, network_name in networks:
    analyze_network(file_path, network_name)



=== Results for network: Enron ===
Number of nodes: 34
Number of edges: 78
Average node degree: 4.59
Number of connected components: 1
Number of nodes in cliques (min_size=4): 12
Number of nodes in stars: 4

=== Results for network: Facebook ===
Number of nodes: 77
Number of edges: 254
Average node degree: 6.60
Number of connected components: 1
Number of nodes in cliques (min_size=4): 48
Number of nodes in stars: 1


| Vlastnost                              | Enron 📧 | Facebook 📘 |
| :------------------------------------- | :------- | :---------- |
| **Počet vrcholů**                      | 34       | 77          |
| **Počet hran**                         | 78       | 254         |
| **Hustota sítě**                       | 0.140    | 0.086       |
| **Průměrný stupeň**                    | 4.59     | 6.60        |
| **Maximální stupeň**                   | -        | -           |
| **Průměrný shluk. koef.**              | -        | -           |
| **Počet komponent**                    | 1        | 1           |
| **Počet vrcholů v klike (min_size=4)** | 12       | 48          |
| **Počet hvězd**                        | 4        | 1           |



## Část 2: Porovnejte výsledky
1. Jaký typ rozložení vrcholů byl použit a proč je pro danou síť vhodný?
---
2. Které vrcholy mají nejvíce hran a jaký význam to může mít pro propojení v síti?

---
3. Kolik má síť vrcholů a hran, jaký je průměrný počet hran na vrchol a co to vypovídá o její struktuře?

- Enron: 34 uzlů, 78 hran, průměrný stupeň 4,59 → řidší síť.

- Facebook: 77 uzlů, 254 hran, průměrný stupeň 6,60 → hustší síť.

---
4. Jak daleko jsou od sebe průměrně jednotlivé vrcholy a co to říká o propojenosti sítě?

- Počet uzlů v klikách (min_size=4): Enron 12, Facebook 48 → skupiny existují.

---
5. Existují v síti skupiny vrcholů, které jsou mezi sebou více propojené?
- Enron: 12 uzlů v klike, Facebook: 48 → Facebook má větší skupiny, což naznačuje výraznější komunity.
---
6. Jsou tyto skupiny tvořené podobným počtem vrcholů nebo se výrazně liší a co by to mohlo znamenat?

  - Enron: 4 „hvězdy“, Facebook: 1 „hvězda“.
  - Kliky a hvězdy ukazují na přítomnost hustě propojených podskupin a centrálních uzlů.
---
7. Které vrcholy propojují různé části sítě a jakou roli v ní mohou hrát?

  - Enron: méně propojená síť.

  - Facebook: více propojená síť, silné komunity.

---
8. Jaké typické struktury lze v síti pozorovat?

---
9. Jak by se změnila síť, pokud by byly odstraněny vrcholy s největším počtem hran?


---
10. Jak na základě propojení vrcholů popsat soudržnost sítě a vztahy mezi jejími částmi?